# Storytelling: Explorando e Modelando os Dados de Preço de Venda de Imóveis


# # Introdução

O objetivo desta análise é entender melhor os fatores que influenciam os preços de venda das casas e desenvolver modelos preditivos para estimar esses preços. Para isso, seguimos várias etapas de análise exploratória de dados (EDA) e modelagem.

# Carregamento e Visualização dos Dados

Começamos carregando os dados do arquivo train.csv e visualizando suas primeiras linhas para entender a estrutura do dataframe. Observamos que os dados contêm várias colunas, cada uma representando uma característica diferente das casas.

In [3]:
import pandas as pd
# Carregar os dados
train_data = pd.read_csv('/mnt/data/train.csv')
# Visualizar as primeiras linhas do dataframe
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Verificação de Valores Ausentes

Identificamos que algumas colunas possuem muitos valores ausentes. As colunas PoolQC, MiscFeature, Alley, Fence e FireplaceQu apresentaram o maior número de valores ausentes.

In [4]:
# Verificar valores ausentes
missing_values = train_data.isnull().sum().sort_values(ascending=False)
missing_values = missing_values[missing_values > 0]
missing_values

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageCond        81
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64

# Estatísticas Descritivas

Analisamos estatísticas descritivas das variáveis numéricas para obter insights sobre sua distribuição e características principais. Observamos que variáveis como SalePrice, OverallQual, GrLivArea, GarageCars e GarageArea apresentaram grande variação, sugerindo sua importância na previsão dos preços.

In [ ]:
# Estatísticas descritivas das variáveis numéricas
desc_stats = train_data.describe()
desc_stats


# Análise de Correlação

Calculamos a correlação entre as variáveis numéricas e SalePrice para identificar quais variáveis têm maior influência sobre o preço de venda. As variáveis OverallQual, GrLivArea, GarageCars, GarageArea, TotalBsmtSF, 1stFlrSF, FullBath, TotRmsAbvGrd, YearBuilt e YearRemodAdd mostraram correlações positivas significativas com SalePrice.

In [ ]:
# Calcular a correlação entre as variáveis numéricas e SalePrice
correlation_matrix = train_data.corr()
saleprice_correlation = correlation_matrix["SalePrice"].sort_values(ascending=False)
saleprice_correlation


# Engenharia de Características

Para preparar os dados para modelagem, removemos colunas com muitos valores ausentes e imputamos os valores ausentes restantes. Selecionamos as variáveis com maior correlação com SalePrice para serem usadas no modelo de regressão.

In [ ]:
# Remover colunas com mais de 50% de valores ausentes
threshold = len(train_data) * 0.5
train_data = train_data.drop(columns=missing_values[missing_values > threshold].index)

# Imputar valores ausentes nas colunas restantes
for column in train_data.columns:
    if train_data[column].isnull().sum() > 0:
        if train_data[column].dtype == "object":
            train_data[column] = train_data[column].fillna(train_data[column].mode()[0])
        else:
            train_data[column] = train_data[column].fillna(train_data[column].median())


# Modelagem: Regressão Linear

Construímos um modelo de regressão linear usando as variáveis selecionadas. Avaliamos o modelo com as métricas de erro quadrático médio (MSE), raiz do erro quadrático médio (RMSE) e o coeficiente de determinação (R²).

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Instanciar o modelo de regressão linear
model = LinearRegression()

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o desempenho do modelo
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

mse, rmse, r2


# Modelagem: Classificação

Para analisar os preços de venda como uma variável categórica (alto ou baixo), construímos um modelo de regressão logística. Avaliamos o desempenho do modelo com métricas como precisão, recall, F1-score e a matriz de confusão.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Converta a variável de saída em uma variável binária (Preço alto ou baixo)
threshold = data['SalePrice'].median()
y_bin = (y > threshold).astype(int)

# Treinar o modelo de regressão logística
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Fazer previsões
y_pred = clf.predict(X_test)

# Calcular as métricas de avaliação
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

# Aprendizagem Não Supervisionada: Clusterização

Aplicamos o algoritmo K-Means para identificar grupos de casas com características semelhantes. Utilizamos o método do cotovelo para determinar o número ideal de clusters e visualizamos os clusters formados.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Selecionar algumas características relevantes para a clusterização
features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath']
X = data[features].dropna()

# Padronizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Determinar o número ideal de clusters usando o método do cotovelo
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

# Plotar o método do cotovelo
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o')
plt.title('Método do Cotovelo')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS')
plt.show()

# Treinar o modelo K-Means com o número ideal de clusters (vamos supor 3 clusters)
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# Adicionar a coluna de clusters ao DataFrame original
data['Cluster'] = pd.Series(clusters, index=X.index)

# Visualizar os clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x='GrLivArea', y='SalePrice', hue='Cluster', data=data, palette='viridis')
plt.title('Clusters de Casas')
plt.xlabel('Área do Terreno')
plt.ylabel('Preço de Venda')
plt.show()


# Redução de Dimensionalidade

Usamos a técnica PCA para reduzir a dimensionalidade dos dados e visualizá-los em um espaço de menor dimensão, facilitando a visualização dos clusters.

In [ ]:
from sklearn.decomposition import PCA

# Aplicar PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Adicionar os componentes principais ao DataFrame original
data['PCA1'] = X_pca[:, 0]
data['PCA2'] = X_pca[:, 1]

# Visualizar os dados em 2D
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PCA1', y='PCA2', hue='Cluster', data=data, palette='viridis')
plt.title('Redução de Dimensionalidade com PCA')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.show()

# Análise de Associação

Usamos o algoritmo Apriori para identificar associações entre as características das casas.

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

# Selecionar algumas características binarizadas para a análise de associação
features_bin = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath']
data_bin = data[features_bin].applymap(lambda x: 1 if x > x.median() else 0)

# Aplicar o algoritmo Apriori
frequent_itemsets = apriori(data_bin, min_support=0.1, use_colnames=True)

# Gerar as regras de associação
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Visualizar as regras de associação
rules.head()


# Análise de Outlier

Utilizamos o algoritmo Local Outlier Factor (LOF) para identificar casas que podem ser consideradas outliers.

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

# Treinar o modelo LOF
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
data['Outlier'] = lof.fit_predict(X_scaled)

# Visualizar os outliers
plt.figure(figsize=(10, 6))
sns.scatterplot(x='GrLivArea', y='SalePrice', hue='Outlier', data=data, palette='viridis')
plt.title('Análise de Outliers com LOF')
plt.xlabel('Área do Terreno')
plt.ylabel('Preço de Venda')
plt.show()


# Conclusão

Através dessas análises, conseguimos identificar as características mais relevantes para a previsão dos preços de venda das casas, construir modelos preditivos e identificar padrões interessantes nos dados. Este processo de EDA e modelagem é essencial para compreender melhor os dados e tomar decisões informadas no desenvolvimento de modelos de machine learning.